In [1]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import IPython

import random 

In [2]:
class Guest(ap.Agent):

    def setup(self):
        # Initialize an attribute with a parameter
        self.random = random.Random()  # Create agent generator
        self.velocity = self.random.randrange(50, 60, 1)
        self.location = self.model.nprandom.random(self.p.ndim)
        self.guest_type = self.random.choice['dancer', 'drinker', 'buyer']
        self.target = None
    
    def setup_pos(self, space):
        self.space = space
        self.pos = space.positions[self]

    def update_position(self, x_pos, y_pos):
        delta_dist = calc_next_pos(self.pos, [x_pos, y_pos], self.velocity)
        self.space.move_by(self, delta_dist)
    
class Stages(ap.Agent):

    def setup(self):
        # Initialize an attribute with a parameter
        self.random = random.Random()  # Create agent generator
        self.locations = [[20, 90], [50, 90], [80, 90], 
                          [20, 50], [80, 50],
                          [50, 50]]
        self.target = None
    
    def setup_pos(self, space):
        self.space = space
        
        
class FestivalModel(ap.Model):

    def setup(self):
        """ Initializes the agents and state space of the model. """
        
        self.space = ap.Space(self, shape=[self.p.size]*self.p.ndim)
        self.guests = ap.AgentList(self, self.p.guests, Guest)
        self.num_stages = self.p.stages
        self.space.add_agents(self.guests, random=True)
        
        for i in range(self.num_stages):
            self.stages = ap.AgentList(self, 1, Stages)
            self.space.add_agents(self.stages, positions=[self.stages.locations[0][i]])
        self.guests.setup_pos(self.space)

    def step(self):
        """
            Defines the models' events per simulation step. 
            Call a method for every agent. 
        """
        x_pos = random.choice([0, 50, 100])
        y_pos = random.choice([0, 50, 100])
        self.guests.update_position(x_pos, y_pos)

    # def update(self):
    #     """ Record a dynamic variable. """
    #     self.agents.record('test')

    def end(self):
        """ Report an evaluation measure. """
        # self.report('my_measure', 1)
        pass

In [3]:
def calc_next_pos(pos, goal, v):
    # dist = np.hypot((goal[0] - pos[0]), (goal[1] - pos[1]))
    return [(goal[0] - pos[0])/v, (goal[1] - pos[1])/v]

In [9]:
def animation_plot_single(m, ax):
    ndim = m.p.ndim
    ax.set_title(f"Festival Simulation {ndim}D t={m.t}")
    
    pos = m.space.positions.values()
    pos = np.array(list(pos)).T
    
    pos1 = pos[:, :-m.num_stages]
    pos2 = pos[:, -m.num_stages:-3]
    pos3 = pos[:, -m.num_stages+3:-m.num_stages+3+2]
    pos4 = pos[:, -1]
    
    # GUESTS
    ax.scatter(*pos1, s=10, c='black', marker='^')
    
    # SHOPS
    ax.scatter(*pos2, s=5000, c='yellow', alpha=0.4, marker='s')
        
    # RESTAURANTS
    ax.scatter(*pos3, s=7000, c='green', alpha=0.4, marker='s')
    
    # CONCERT
    ax.scatter(*pos4, s=11000, c='blue', alpha=0.4, marker='o')
    
    ax.set_xlim(0, m.p.size)
    ax.set_ylim(0, m.p.size)
    
    if ndim == 3:
        ax.set_zlim(0, m.p.size)
    ax.set_axis_off()

def animation_plot(m, p):
    projection = '3d' if p['ndim'] == 3 else None
    fig = plt.figure(figsize=(15,10))
    fig.patch.set_facecolor('white')
    ax = fig.add_subplot(111, projection=projection)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20))

In [10]:
parameters = {
    'guests': 50,
    'steps': 100,
    'size': 100,
    'stages': 6, 
    'ndim': 2,
    'seed': 42
}

model = FestivalModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.062372
Simulation finished


In [11]:
animation_plot(FestivalModel, parameters)